In [25]:
import pandas as pd
import numpy
import pprint

In [26]:
# df1 = pd.read_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\koethen-test.csv')
# df2 = pd.read_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\koethen-test-acc.csv')

# df1['timestamp [ms]'] = pd.to_datetime(df1['timestamp [ms]'], unit='ms')
# df2['timestamp [ms]'] = pd.to_datetime(df2['timestamp [ms]'], unit='ms')

# merged_df = pd.merge(df1, df2, on=[
#                      'deviceId', 'measurementId', 'trackId', 'timestamp [ms]'], how='outer')

# merged_df = merged_df.sort_values(by='timestamp [ms]')
# merged_df['latitude'] = merged_df['latitude'].fillna(method='ffill')
# merged_df['longitude'] = merged_df['longitude'].fillna(method='ffill')

# merged_df['accuracy [m]'] = merged_df['accuracy [m]'].fillna(method='ffill')
# merged_df.to_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\koethen-test-both.csv')

In [27]:
# df_location = pd.read_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\koethen-test.csv')
# df_acceleration = pd.read_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\koethen-test-acc.csv')

In [28]:
df_location = pd.read_csv(
    r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\2024-01-08_Koethen\locations_2024-01-08_15-17-10.csv')
df_acceleration = pd.read_csv(
    r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\2024-01-08_Koethen\acceleration_2024-01-08_16-12-23.csv')

In [29]:
df_location['timestamp [ms]'] = pd.to_datetime(
    df_location['timestamp [ms]'], unit='ms')
df_acceleration['timestamp [ms]'] = pd.to_datetime(
    df_acceleration['timestamp [ms]'], unit='ms')

df_location.sort_values(by='timestamp [ms]', inplace=True)
df_acceleration.sort_values(by='timestamp [ms]', inplace=True)

In [30]:
def interpolate_lat_lon(acc_row, df_location):
    # Find the nearest location timestamps before and after the acceleration measurement
    before = df_location[df_location['timestamp [ms]']
                         <= acc_row['timestamp [ms]']].tail(1)
    after = df_location[df_location['timestamp [ms]']
                        > acc_row['timestamp [ms]']].head(1)

    if before.empty or after.empty:
        return np.nan, np.nan  # Return NaN if no bounding GPS data points are found

    # Linear interpolation
    total_time_diff = (after.iloc[0]['timestamp [ms]'] -
                       before.iloc[0]['timestamp [ms]']).total_seconds()
    elapsed_time = (acc_row['timestamp [ms]'] -
                    before.iloc[0]['timestamp [ms]']).total_seconds()

    lat_interp = before.iloc[0]['latitude'] + (
        after.iloc[0]['latitude'] - before.iloc[0]['latitude']) * (elapsed_time / total_time_diff)
    lon_interp = before.iloc[0]['longitude'] + (
        after.iloc[0]['longitude'] - before.iloc[0]['longitude']) * (elapsed_time / total_time_diff)

    return lat_interp, lon_interp


# Apply the interpolation function to each row in the acceleration dataframe
df_acceleration[['interp_latitude', 'interp_longitude']] = df_acceleration.apply(
    lambda row: interpolate_lat_lon(row, df_location), axis=1, result_type='expand')

In [ ]:
df_acceleration.to_csv(
    r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\2024-01-08_Koethen\koethen-test-both.csv')

In [ ]:
df_acceleration.dropna(
    subset=['interp_latitude', 'interp_longitude'], inplace=True)